# Create Your Own Data-Connected Assistant - DivaConf2024
## Gemini API: Function calling with Python

![Diva: Dive into AI](https://cdn.prod.website-files.com/6645f6ac57776cd5f3e4ac96/66633085e40eb51b599aedf3_indigo_soft_2.png "Diva: Dive into AI")

https://www.divaconf.com/

**Install & Import Libraries**

In [ ]:
!pip install google-generativeai==0.7.2

In [1]:
import google.generativeai as genai
import requests
import json
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [8]:
GOOGLE_API_KEY = "{GOOGLE AI STUDIO - API KEY}"

genai.configure(api_key=GOOGLE_API_KEY)

## DEMO 1 - Döviz Kuru Asistanı

**Frankfurter** is an open-source API for current and historical foreign exchange rates published by the European Central Bank.

In [85]:
# Docs: https://www.frankfurter.app/docs/

url = f"https://api.frankfurter.app/latest?from=USD&to=TRY"

response = requests.get(url)

if response.status_code == 200:
  data = response.json()
  print(data)
  print(data["rates"])

{'amount': 1.0, 'base': 'USD', 'date': '2024-07-12', 'rates': {'TRY': 33.027}}
{'TRY': 33.027}


**Define Python functions and a function handler**

Define Python functions that you'll invoke to fetch data from an external API:

In [86]:
def get_exchange_rate(currency_to: str, currency_from: str):
    """
    Get the exchange rate for currencies between countries
    """

    url = f"https://api.frankfurter.app/latest?from={currency_from}&to={currency_to}"

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return "Failed exchange rate!"


**Initialize model**

Initialize the Gemini model with the desired model parameters and Tool that we defined earlier:

In [93]:
model = genai.GenerativeModel(model_name="gemini-1.5-pro",
                              tools=[get_exchange_rate]
                              )

In [94]:
model

genai.GenerativeModel(
    model_name='models/gemini-1.5-pro',
    generation_config={},
    safety_settings={},
    tools=<google.generativeai.types.content_types.FunctionLibrary object at 0x79ec82d5b220>,
    system_instruction=None,
    cached_content=None
)

**Initialize chat session**

In [95]:
chat = model.start_chat(enable_automatic_function_calling=True)

In [99]:
prompt = "5 bin yuro kaç lira eder?"

In [100]:
from IPython.display import display, Markdown

#Send Message
response = chat.send_message(prompt)

#Print Result
display(Markdown(response.text))

5 Bin Euro, bugun itibariyle 179.830 Türk Lirası ediyor. 


In [101]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print('-'*80)

user -> [{'text': '5bin yuro kaç lira eder?'}]
--------------------------------------------------------------------------------
model -> [{'text': 'I can\'t answer that question. I need to know which currencies "bin" and "lira" refer to. \n'}]
--------------------------------------------------------------------------------
user -> [{'text': '5 bin yuro kaç lira eder?'}]
--------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'get_exchange_rate', 'args': {'currency_to': 'TRY', 'currency_from': 'EUR'}}}]
--------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'get_exchange_rate', 'response': {'date': '2024-07-12', 'base': 'EUR', 'amount': 1.0, 'rates': {'TRY': 35.966}}}}]
--------------------------------------------------------------------------------
model -> [{'text': '5 Bin Euro, bugun itibariyle 179.830 Türk Lirası ediyor. \n'}]
------------------------------

## DEMO 2 - Podcast Asistanı

**Define Python functions and a function handler**

Define Python functions that you'll invoke to fetch data from an external API:

In [128]:
TALKTRACK_API_TOKEN = "{SERVICE TOKEN}"
TALKTRACK_API_URL_ROOT = "https://podcast-graph-app-api.vercel.app/api"


def api_service_all_podcasts():
  """Finds all podcast information in the application.

    Args:
      None

    Returns: In the podcast list;
    - podcast_id
    - podcast_name
    - podcast_publisher
    - podcast_description
    - podcast_image_path
    - podcast_feed_url
    - podcast_website
    - podcast_cover_color
    - podcast_updated_timestamp
    - podcast_updated_date
    - podcast_total_episode_count
    - podcast_genre_primary
  """

  url = f"{TALKTRACK_API_URL_ROOT}/podcasts/"

  headers = {"token": f"{TALKTRACK_API_TOKEN}"}

  response = requests.request("GET", url, headers=headers)

  #print(response.status_code)
  #print(response.text)

  message = ""
  status_code = response.status_code

  if response.status_code == 200:
    records = json.loads(response.text)
    #print(json.dumps(records, indent=4))

    return records
  else:
    result = json.loads(response.text)
    error_message = result["message"]
    error_code = result["code"]

    message = error_message

  return {"status_code": status_code, "message": message}


def last_episodes_by_podcast_id(podcast_id: str):
  """Search last episodes by podcast id

  Args:
        podcast_id: The unique ID of the podcaster obtained from the entire podcast query.

  Returns: In addition to podcast information, episode information;
  - episode_id
  - episode_guid
  - episode_audio_url
  - episode_name
  - episode_summary
  - episode_description
  - episode_duration
  - episode_duration_timestamp
  - episode_published_timestamp
  - episode_published_date
  - podcast_id
  - podcast_name
  - is_processed_data
  """

  url = f"{TALKTRACK_API_URL_ROOT}/episodes/last_episodes_by_podcast_id"

  payload = {
    "podcast_id": podcast_id,
    "limit": 10
  }

  headers = {"token": f"{TALKTRACK_API_TOKEN}"}

  response = requests.request("POST", url, json=payload, headers=headers)

  #print(response.status_code)
  #print(response.text)

  message = ""
  status_code = response.status_code

  if response.status_code == 200:
    records = json.loads(response.text)
    #print(json.dumps(records, indent=4))

    return records
  else:
    result = json.loads(response.text)
    error_message = result["message"]
    error_code = result["code"]

    message = error_message

  return {"status_code": status_code, "message": message}


Define a function handler that maps function call names (from your function declarations) to actual Python functions that call APIs:


In [129]:
functions = {
    "api_service_all_podcasts": api_service_all_podcasts,
    "last_episodes_by_podcast_id": last_episodes_by_podcast_id,
}

**Initialize model**

Initialize the Gemini model with the desired model parameters and Tool that we defined earlier:

In [119]:
#generation_config = {
#    "temperature": 0
#}

#model = genai.GenerativeModel(model_name="gemini-1.5-flash",
#                              tools=functions.values(),
#                              generation_config=generation_config
#                              )

model = genai.GenerativeModel(model_name="gemini-1.5-pro",
                              tools=functions.values()
                              )

**Auto Function Calling**

In [120]:
chat = model.start_chat(enable_automatic_function_calling=True)

In [122]:
prompt = "Nasıl Olunur'un son 3 bölümü nedir?"

response = chat.send_message(prompt)
display(Markdown(response.text))

Nasıl Olunur'un son 3 bölümü:

1. **221- Nedim Saban:** (2 saat 11 dakika) 
Yayınlanma tarihi: 11 Temmuz 2024
2. **220- Ezgi Mola:** (1 saat 31 dakika) 
Yayınlanma tarihi: 25 Haziran 2024
3. **219- Ayşegül Aldinç:** (1 saat 13 dakika) 
Yayınlanma tarihi: 12 Haziran 2024 


In [123]:
prompt = "Kırılma Noktası adlı podcaste son 5 bölümdeki konuk isimleri kimlerdir?"
response = chat.send_message(prompt)
display(Markdown(response.text))

Kırılma Noktası podcastinin son 5 bölümündeki konukların isimleri şunlardır:

1. Bahar Şevket
2. Umut Şirin
3. Nur Yeşilyurt Güçlü 
4. Aydan Taşdemir
5. Burak Şentürk 


In [124]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print('-'*80)

user -> [{'text': "Nasıl Olunur'un son 3 bölümü nedir?"}]
--------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'api_service_all_podcasts', 'args': {}}}]
--------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'api_service_all_podcasts', 'response': {'podcasts': [{'podcast_id': '1505264003', 'podcast_cover_color': '#00BC5C', 'podcast_publisher': 'M. Serdar Kuzuloğlu', 'podcast_genre_primary': 'TECH', 'podcast_image_path': 'https://is3-ssl.mzstatic.com/image/thumb/Podcasts113/v4/af/d8/ac/afd8ac9e-1485-43b6-5f5a-3be0701ba4a7/mza_4069295067848953951.jpeg/640x640bb.png', 'podcast_total_episode_count': None, 'podcast_feed_url': 'https://feeds.podcastmirror.com/zihnimin-kivrimlari', 'podcast_name': 'Zihnimin Kıvrımları', 'podcast_website': 'https://www.mserdark.com/zihnimin-kivrimlari/', 'podcast_description': 'Hayatımızın her yanını kaplayan teknoloji bizim isteği

In [126]:
chat = model.start_chat(enable_automatic_function_calling=True)

prompt = "Bi' Gidene Soralım podcastinin 150. Bölüm'ün özetini yapar mısın?"

response = chat.send_message(prompt)
display(Markdown(response.text))

Bi' Gidene Soralım podcastinin 150. bölümünde, Masterchef yarışmasından tanıdığımız Somer Sivrioğlu konuk oluyor. Bölümde Somer Şef'in 1995 yılında yüksek lisans yapmak için Avustralya'ya taşınması ve orada kurduğu Türk restoranı Efendy'nin başarısı anlatılıyor. Somer Şef zamanla Anason, Baharat ve Maydanoz gibi başka restoranlar da açarak Türk mutfağını Avustralya'da tanıtmış. Ayrıca "Anadolu" adlı yemek kitabı ve Masterchef Avustralya jüriliği gibi çalışmalarıyla tanınıyor. Bölümde Somer Bey'in Avustralya'daki yaşamı, Türk ve Avustralya yemek sektörleri arasındaki farklar ve benzerlikler ve kariyeri ele alınıyor. 


In [127]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print('-'*80)

user -> [{'text': "Bi' Gidene Soralım podcastinin 150. Bölüm'ün özetini yapar mısın?"}]
--------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'api_service_all_podcasts', 'args': {}}}]
--------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'api_service_all_podcasts', 'response': {'podcasts': [{'podcast_updated_timestamp': None, 'podcast_image_path': 'https://is3-ssl.mzstatic.com/image/thumb/Podcasts113/v4/af/d8/ac/afd8ac9e-1485-43b6-5f5a-3be0701ba4a7/mza_4069295067848953951.jpeg/640x640bb.png', 'podcast_feed_url': 'https://feeds.podcastmirror.com/zihnimin-kivrimlari', 'podcast_total_episode_count': None, 'podcast_genre_primary': 'TECH', 'podcast_cover_color': '#00BC5C', 'podcast_updated_date': None, 'podcast_id': '1505264003', 'podcast_description': 'Hayatımızın her yanını kaplayan teknoloji bizim isteğimizle mi şekilleniyor yoksa o mu bizi şekillendiriyor? A